In [2]:
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Define the model architecture
model = Sequential([
    Flatten(input_shape=(200, 200, 3)),  # Flatten the input image to a one dimensional vector
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(24, activation='softmax')
])

# Compile the model with an Adam optimizer and Sparse Categorical Cross Entropy loss function
model.compile(optimizer=Adam(),
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
# Dataset: https://www.kaggle.com/datasets/shivajbd/jigsawpuzzle
# Load the Jigsaw Puzzle dataset and split it into train, test and valid sets
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define function to load dataset from csv files and images
def load_dataset(root_dir, csv_filename):
    data = pd.read_csv(csv_filename)
    images = []
    labels = []
    for index, row in data.iterrows():
        image_path = os.path.join(root_dir, row['image'])
        image = load_img(image_path, target_size=(200, 200))
        image = img_to_array(image) / 255.0
        images.append(image)
        labels.append([int(x) for x in row['label'].split()])  # Convert string label to list of integers
    return images, labels

# Load data for 2x2 puzzle
train_images_2x2, train_labels_2x2 = load_dataset('puzzle_2x2/train', 'puzzle_2x2/train.csv')
test_images_2x2, test_labels_2x2 = load_dataset('puzzle_2x2/test', 'puzzle_2x2/test.csv')
valid_images_2x2, valid_labels_2x2 = load_dataset('puzzle_2x2/valid', 'puzzle_2x2/valid.csv')

# Load data for 3x3 puzzle
train_images_3x3, train_labels_3x3 = load_dataset('puzzle_3x3/train', 'puzzle_3x3/train.csv')
test_images_3x3, test_labels_3x3 = load_dataset('puzzle_3x3/test', 'puzzle_3x3/test.csv')
valid_images_3x3, valid_labels_3x3 = load_dataset('puzzle_3x3/valid', 'puzzle_3x3/valid.csv')

# Concatenate 2x2 and 3x3 datasets
train_images = train_images_2x2 + train_images_3x3
train_labels = train_labels_2x2 + train_labels_3x3
test_images = test_images_2x2 + test_images_3x3
test_labels = test_labels_2x2 + test_labels_3x3
valid_images = valid_images_2x2 + valid_images_3x3
valid_labels = valid_labels_2x2 + valid_labels_3x3

# Split dataset into train, test, and validation sets
train_data, test_data, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)
train_data, valid_data, train_labels, valid_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Convert lists to numpy arrays
train_data = np.array(train_data)
test_data = np.array(test_data)
valid_data = np.array(valid_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
valid_labels = np.array(valid_labels)

# Print shapes of the datasets
print("Train data shape:", train_data.shape)
print("Train labels shape:", train_labels.shape)
print("Test data shape:", test_data.shape)
print("Test labels shape:", test_labels.shape)
print("Validation data shape:", valid_data.shape)
print("Validation labels shape:", valid_labels.shape)

In [ ]:

# Normalize pixel values to [0, 1]
train_data = train_data / 255.0
test_data = test_data / 255.0

# Train the model
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data, test_labels)
print("Test accuracy:", test_acc)

# Make predictions
predictions = model.predict(test_data)